1. 각 월별 곰스 파일을 읽어온다  
2. 데이터 전처리   
  - 계약상태를 해당월 말일 기준으로
  - 메트로 산하 지점 코드 => 실코드로 전환  (지점공란은 엠피메트로, 직할지점 HL지점, 씨티엠지점,제이엔제이는 지점으로 승격, 팀 시흥1팀은 엠피시흥)
  - 직영,지사 구분값 
  - 계약일자가 해당월의 몇번째 영업일인지 생성

3. 시각화: 전체/ 직영 구분 plot
  - 당월,전년동기 라인 그래프
  - 최근 4개월  라인그래프
  - 당월 일자별 막대그래프 

## 1.기초 데이터 일기 및 전처리

In [1]:
import pandas as pd
import mypkgs.MyDateUtil as MyU  #자체 유틸리티 클래스 로딩
import mypkgs.GladUtil as glad

from datetime import date, datetime
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
outdir='d:/글로벌/실적분석/'  #그래르를 저장할 위치

data=pd.read_excel('d:/글로벌/실적분석/코드파일.xlsx',sheet_name=None)
#data=pd.read_excel('e:/python/testData/코드파일.xlsx',sheet_name=None)

df_Files=data['분석파일']
# df_Conv=data['지점변환']
# df_Grp=data['지점그룹']
df_Fc=data['사원']

getDT = MyU.getDateUtil()

strTodate= lambda x: date(int(dt.split('-')[0]), int(dt.split('-')[1]), int(dt.split('-')[2]) )
firstDate= lambda x: datetime(x.year,x.month,1) #해당월의 1일

dfList=[pd.DataFrame()]*len(df_Files)
def preProc(dfT,Method):
    dfT= dfT[(dfT.NO!='합계') & (dfT.계약종류.isin(['장기','생보']))]   #불필요 데이터 삭제
    
    dfT.rename(columns={'납입주기(방법)':'납입방법'}, inplace=True)
    temp = dfT.loc[dfT['납입방법']=='년납','초회보험료'] / 12
    dfT.loc[dfT.납입방법=='년납','초회보험료'] = temp
    del temp
    temp = dfT.loc[dfT['납입방법']=='일시납','초회보험료'] / dfT.loc[dfT.납입방법=='일시납','납입기간']
    dfT.loc[dfT.납입방법=='년납','초회보험료'] = temp
    del temp
    
    dfT['계약일자']=pd.to_datetime(dfT.계약일자,format='%Y-%m-%d')
    dfT['최종상태변경일']=pd.to_datetime(dfT.최종상태변경일,format='%Y-%m-%d')
    dfT['말일']=pd.to_datetime(dfT.계약일자.map(getDT.get_lastday))
     
    dfT['월말계약상태']= dfT.계약상태
    dfT.loc[dfT.말일 < dfT.최종상태변경일,'월말계약상태']='정상'
    if Method=='계약월':
        dfT=dfT[dfT.월말계약상태=='정상']  #계약월 기준인 경우 계약월말 정상건만 추출
 
    dfT=glad.BrhConv(dfT,'지점그룹')  #지점그룹 생성 / 메트로 코드 분리
    dfT['영업일차']=dfT.계약일자.map(getDT.get_workdaysMonth)
#    dfT['영업일차']=dfT.apply(lambda x: getDT.get_workdays(firstDate(x['계약일자']),x['계약일자']),axis=1)
    dfT['월']= dfT.계약일자.map(lambda x: datetime.strftime(x,"%Y-%m")[2:]+'월')
    return dfT


dfAll=[]
dfAll=pd.DataFrame()
maxDay=0  #최대영업일수
##각 파일별 df를 list
for seq, row in df_Files.iterrows():
    print(row[0],'-- Processing')
    df=pd.read_excel(row[1]+row[2])
    df= preProc(df,'계약월')
    df= df.reset_index().drop(columns='index')
    dfList[seq]=df
    maxDay=max(maxDay, df.영업일차.max()) 
    #   그래프 그리는 로직 필요
    
maxDay += 1    
#내근직 및 퇴사자 제거
df_Fc=df_Fc[df_Fc.퇴사일자.isna()]   
df_Fc=df_Fc[~df_Fc.업무구분.isin(['내근직'])]
df_Fc=df_Fc[(df_Fc.생보등록일.notnull() & df_Fc.생보말소일.isnull()) | (df_Fc.손보등록일.notnull() & df_Fc.손보말소일.isnull())]
df_Fc=glad.BrhConv(df_Fc,'지점그룹') 
df_Fc=df_Fc.loc[:,['지점그룹','지점','사원번호','사원명']]


20.05월 -- Processing
20.04월 -- Processing
20.03월 -- Processing
20.02월 -- Processing
19.05월 -- Processing


In [20]:
# df = pd.pivot_table(dfList[0], values='초회보험료',
#                        index=['지점그룹', '지점'],
#                        columns=['월','계약종류'],
#                        fill_value=0, aggfunc=sum, dropna=True, ).sort_index(axis=1,ascending=False
df=df.reset_index().sort_values(by=['지점그룹','지점'],ascending=[False,True])

In [23]:
df['지점','']

9        엠피라온
10      엠피레전드
11      엠피메트로
12       엠피성공
13       엠피송내
14     엠피아이리치
15       엠피여주
16      엠피의정부
0       씨티엠지점
1        엠피가온
2        엠피경기
3        엠피상동
4        엠피시흥
5     엠피제이엔제이
6        엠피조은
7       엠피프라임
8      엠피피스토스
Name: (지점, ), dtype: object

In [ ]:
# cols=['월']
cols=['월']
# df=pd.concat([dfList[0],dfList[1],dfList[2],dfList[3],dfList[4]])
df=pd.concat([dfList[0],dfList[1]])
df['초회보험료']=round(df.초회보험료/1000,1)
dd=pd.pivot_table(df, values='초회보험료',
                       index=['계약종류','보험사'],
                       columns=cols,
                       fill_value=0, aggfunc=sum,dropna=True, ).sort_index(axis=1,ascending=False)
dd=pd.concat([d.append(d.sum().rename((k, 'Total'))) for k, d in dd.groupby(level=0)])
dd=pd.concat([dd.loc['생보',:].rename(columns=lambda x: '1'+x).reset_index().rename(columns={'보험사':'생보'}),
        dd.loc['장기',:].rename(columns=lambda x: '2'+x).reset_index().rename(columns={'보험사':'장기'})],axis=1)


In [ ]:
cols='월'
maxDay=dfList[0].영업일차.max()
df=pd.concat([dfList[0],dfList[1],dfList[2],dfList[3]])
d1=df[(df.지점그룹=='직영') &  (df.영업일차 <= maxDay)]
# d1=df[(df.영업일차<= maxDay) ]
dd=pd.pivot_table(d1, values='초회보험료',
                       index=['지점','수금사원명'],
                       columns=cols,
                       fill_value=0, aggfunc=sum,dropna=True, ).sort_index(axis=1,ascending=False)

In [ ]:
col=[]
dd.columns.map(lambda x: col.append(x))
cc=col[:]
idx=-1
for i,k in enumerate(cc):
    if i < len(col)-1:
        newcol= k+'g'
        print(k,'=',newcol)
        idx= idx + 2  #중간에 컬럼삽입
        dd.insert(idx, newcol, dd[col[i]]-dd[col[i+1]])
col=[]
dd.columns.map(lambda x: col.append(x))
d1=dd.sort_values(by=col[0],ascending=False)[col[0]].head(20)
d2=dd.sort_values(by=col[1],ascending=False)[col[1]].head(20)
d3=dd.sort_values(by=col[1],ascending=True)[col[1]].head(20)

# dd.sort_values(by='20-05월',ascending=False)
# dd.sort_values(by='gr',ascending=False)
# dd.sort_values(by='gr')
# dfList[0].영업일차.max()

In [ ]:
dl

In [ ]:
# dd.loc['t']=dd.sum()
# dd.loc['Row_Total1'] = dd.sum(numeric_only=True, axis=0)
# dd.sum(numeric_only=True, axis=0)
# dd.append(dd.sum().rename('Total')).assign(Total=lambda d: d.sum(1) 
dd=pd.concat([d.append(d.sum().rename((k, 'Total'))) for k, d in dd.groupby(level=0)])
dd=pd.concat([dd.loc['생보',:].reset_index(),dd.loc['장기',:].reset_index()],axis=1)
dd
# pd.concat([d.append(d.sum().rename((k, 'Total'))) for k, d in dd.groupby(level=0)])

In [ ]:
# for i in col:
#     d1=dd.head(10).style.format({i: "{:,.0f}"})
print(i in list(col))

In [ ]:
import numpy as np
# df=pd.DataFrame(dfList[0].groupby(['지점그룹','지점','월'])['초회보험료'].agg('sum'))
# df2=pd.DataFrame(dfList[1].groupby(['지점그룹','지점','월'])['초회보험료'].agg('sum'))
# df=pd.concat([df,df2])
# df=df.pivot_table( index=['지점', '지점그룹'],columns='월',values='초회보험료')
# df=df.reset_index()
# dft=pd.DataFrame(df[col].sum(axis=0)).transpose()
# dft['지점그룹']='total'
# dft['지점']=''

# # df.sort_values(by=['지점그룹','지점'],ascending=[False,True])
# # df
# #subtotal
# df=df.groupby(['지점그룹']).apply(lambda sub_df: sub_df.pivot_table(index=['지점'], 
#                                 values=['20-03월','20-04월'], aggfunc='sum', margins=True)).reset_index()
# df
# pd.DataFrame(df.groupby('지점그룹')['20-03월','20-04월'].agg('sum'))


# dft['지점']='total'
# df_jik=pd.DataFrame(df[df.지점그룹=='직영'].groupby(['지점그룹'])[col].sum()).reset_index()
df_jik= df[(df.지점그룹=='직영') & (df.지점=='All') ]

# df_jik2=pd.DataFrame(df_jik[col].apply(lambda x: round(x*100/dft[col]), axis=1))
# df.append(pd.DataFrame(df_jik2))
# for idx,val in ss.items():
#     print(val)
#     print(sum(ss.values))
#     txt = txt + idx + ' ' + '{0:,.0f}'.format(val/1000) + ' ({0:.0f}%)'.format(val/sum(ss.values)*100)


In [ ]:
# pivot
# table = pd.pivot_table(dt, values='초회보험료',
#                        index=['지점그룹', '지점'],
#                        columns=['월'],
#                        fill_value=0, aggfunc=sum, dropna=True, )
# #subtotal grandtotal
# s=pd.concat([
#     d.append(d.sum().rename((k, 'Total')))
#     for k, d in table.groupby(level=0)
# ]).append(table.sum().rename(('Grand', 'Total')))

In [ ]:
def dfMake():
    월=list(range(0,maxDay))
    일차=list(range(1,maxDay+1))
    실적= [0]* maxDay
    누계= [0]* maxDay
    dfd=pd.DataFrame()
    dfd=pd.DataFrame({'월':월,'일차':일차,'실적':실적,'누계':누계})
    return dfd

def workDates(dt):
    wd1=date(dt.year,dt.month,1)
    wd2=getDT.get_lastday(wd1)
    return getDT.get_workdates(wd1,wd2)

dfSum=pd.DataFrame()
workDateList=[] #당월의 영업일 list
thisMonth= dfList[0].월.max()  #가장 최근월을 당월로 설정
prevMonth=''

# 각 df 반복
for seq, dfRow in enumerate(dfList):
#     print(dfRow.월[0],len(dfRow),' 건--processing')
    df_Temp = pd.DataFrame(dfRow.groupby(['월','영업일차'])['초회보험료'].agg('sum')).reset_index()  #집계
    df_Temp['초회보험료']=round(df_Temp['초회보험료']/ 1000,0)
    df_Temp['csum'] = df_Temp.groupby(['월'])['초회보험료'].cumsum()  #누계생성
    dfd = dfMake()
    dfd['월']=df_Temp.월[0]   #컬럼 초기화
    dfd['실적']=0
    dfd['누계']=0
    for seq1, row in enumerate(df_Temp.iterrows()):    # groupby df를 dfSum에 복사
        dfd.iloc[seq1]=[row[1][0],row[1][1],row[1][2],row[1][3]]
    
    dfd['누계']=  dfd.실적.expanding().sum()  #빈 row의 값을 채움
#     dfd['실적']=  dfd.누계.expanding().sum()
    if dfRow.월[0]== thisMonth:
        workDateList=workDates(dfRow.계약일자[0])
        dfThis= dfd
    elif dfRow.월[0] > prevMonth:
        prevMonth=dfRow.월[0]
        
    dfSum=pd.concat([dfSum,dfd])
    del dfd
    

## 월별 실적 추세

In [ ]:

# Window 의 한글 폰트 설정
import matplotlib.font_manager as fm

fm.get_fontconfig_fonts()
font_location = 'c:/windows/Fonts/gulim.ttc'
font_name = fm.FontProperties(fname=font_location).get_name()

fontprop = fm.FontProperties(fname="./MalgunGothic.ttf", size=24)
font = {'family' : font_name,
        'weight' : 'bold',
        'size'   : 16}
plt.rc('font', **font)  # pass in the font dict as kwargs

fontprop = fm.FontProperties(fname=font_location, size=24)

plt.rc('axes', unicode_minus=False)
plt.figure(figsize=(20,10))
# plt.set_size_inches(20, 10)
# sns.set(font_scale=1.0)  sns를 사용하면 한글깨짐
# sns.set_style("white")

plt.rcParams["font.family"] = font_name
plt.rcParams['font.size'] = 12.
plt.rcParams['xtick.labelsize'] = 12.
plt.rcParams['ytick.labelsize'] = 12.
plt.rcParams['axes.labelsize'] = 13. #축 라벨 
plt.rcParams['axes.grid'] = True 

plt.style.context('dark_background')
palette = sns.color_palette("muted", len(dfSum.월.unique())) #각 월 컬러지정

#그래프가 밀리는 현상 제거용
dfSum['s일차']=dfSum.일차 - 1
dfThis['s일차']=dfThis.일차 - 1

mx= dfThis[dfThis.실적>0].index.max()  #당월최종영업일
s=dfSum.shape[0]/(len(dfSum.월.unique()))   #각월의 최종일
plt.title("월별 실적 추세",fontproperties=fontprop,fontdict=font)
g=sns.lineplot(x='s일차', y="누계", hue='월',linewidth=2, data=dfSum,palette=palette)  #월별 실적 line 그래프
g=sns.lineplot(x='s일차', y="누계",linewidth=5,color=palette[0],marker='*', data=dfThis)
for i,val in enumerate(dfSum.누계):  #당월 마지막 영업일, 각월의 최종일에 누계표시
    if  i==mx:
        g.text(x=i,y=round(val),s='{:,d}'.format(int(val)),ha='center')
    if  (i+1) %s==0:
        g.text(x=s-0.5,y=round(val),s='{:,d}'.format(int(val)),ha='center')
        
g=sns.barplot(x="s일차", y="실적",  data=dfThis)   #당월 bar
for i,val in enumerate(dfThis.실적):
    g.text(x=i,y=round(val),s='{:,d}'.format(int(val)),ha='center')

dfSum.drop(columns='s일차',inplace=True)
dfThis.drop(columns='s일차',inplace=True)

txt=''
ss=dfList[0].groupby(['지점그룹'])['초회보험료'].sum()
for idx,val in ss.items():
    txt = txt + idx + ' ' + '{0:,.0f}'.format(val/1000) + ' ({0:.0f}%)'.format(val/sum(ss.values)*100)
    txt = txt + '\n'
txt = txt + '합계' + ' ' + '{0:,.0f}'.format(sum(ss.values)/1000) + '  ♠~♠'
txt=txt.rstrip()    
plt.text( s/3, dfSum.누계.max()*0.9, txt, bbox=dict(boxstyle='square', color='yellowgreen'),fontsize=20)

plt.ylabel("누계",rotation=45)
plt.xlabel("영업일차")
plt.xticks(range(0,23),workDateList,rotation = 45)
plt.legend()
plt.savefig(outdir+'report1.png',dpi=120)
plt.show()

In [ ]:
# dfList[0].계약일자.map(lambda x: x.strftime('%Y-%m-%d'))
# df=df[df.계약종류.isin(['장기','생보'])]   #불필요 데이터 삭제
workDateList

## 지점별 실적
  - 히트맵

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2, sharey=False)
plt.subplots_adjust(wspace=.8)
plt.tight_layout()
fig.set_size_inches(20, 10)

fp=dfList[0].groupby(['지점','계약일자'])['초회보험료'].count().reset_index()
fp['일자']= fp.계약일자.map(lambda x: datetime.strftime(x,'%d'))
fp1 = fp.pivot('지점','일자','초회보험료').fillna(0)
g=sns.heatmap(fp1, annot=True, fmt='.3g', cmap=sns.light_palette('red'),ax=ax1)
g.set_title('일자별 건수')
g.set_xlabel("Date",rotation=45)
g.set_ylabel("Branch",rotation=60)

fp=dfList[0].groupby(['지점','계약일자'])['초회보험료'].sum().reset_index()
fp['일자']= fp.계약일자.map(lambda x: datetime.strftime(x,'%d'))
fp['초회보험료']=round(fp['초회보험료']/10000,0)
fp1 = fp.pivot('지점','일자','초회보험료').fillna(0)
g=sns.heatmap(fp1, annot=True, fmt='.3g', cmap=sns.light_palette('blue'),ax=ax2)
g.set_title('일자별 실적(만원)')
g.set_xlabel("Date",rotation=45)
g.set_ylabel("Branch",rotation=60)
plt.savefig(outdir+'report2.png',dpi=120)

In [ ]:
##sns 두개를 하나에 그리기
fig,(ax1,ax2) = plt.subplots(1,2, sharey=False)
plt.subplots_adjust(wspace=.8)

fig.set_size_inches(20, 10)
# print(fig.get_size_inches)

#당월건 분석
dfT=dfList[0].loc[dfList[0].월==thisMonth]
dfT=pd.pivot_table(dfT,index=['지점그룹','지점'],columns='계약종류',values='초회보험료'
     ,aggfunc=sum,fill_value=0).reset_index()
# if not dfT.columns.contains('생보'):  #생보없으면 생성
#     dfT['생보']=0
    
dfT['합']=dfT.장기+dfT.생보
dfT.iloc[:,2:5]=dfT.iloc[:,2:5]/1000  #천원단위
df1= dfT

# 1번 그래프 : 지점별 실적
ax1.bar(df1.지점, df1.합,color='g',label='생보')
ax1.bar(df1.지점, df1.장기,color='blue',label='장기')
ax1.set_xticklabels(df1.지점, rotation=-45)
ax1.set_title('실적-지점/구성비',fontdict=font)
ax1.set_xlabel("지점")
ax1.set_ylabel("실적",rotation=45)
ax1.legend()

# 2번 그래프 : 직영사원 top10
dfT=dfList[0].loc[dfList[0].월==thisMonth]
df1=dfT[dfT.지점그룹=='직영'].groupby(['지점','수금사원명'])['초회보험료'].sum().reset_index().sort_values(by='초회보험료',ascending=False).head(20)
df1['초회보험료']=df1.초회보험료 /1000
g= sns.barplot(x="수금사원명", y="초회보험료",data=df1, ax = ax2)
g.set_xticklabels(g.get_xticklabels(), rotation=45,
                  horizontalalignment='right',
                  fontweight='light',
                  fontsize='11'
                 )
ax2.set_title('직영사원Top20',fontdict=font)
ax2.set_ylabel("실적",rotation=45)
ax2.set_xlabel("X=우수사원")
# ax2.legend()

plt.tight_layout()
plt.savefig(outdir+'report3.png',dpi=120)

In [ ]:
# dd=pd.DataFrame(dfT.지점.unique(),columns=['지점'])
dd=pd.concat([dfList[0],dfList[1]])
dfT=pd.pivot_table(dd,index=['지점그룹','지점'],columns='계약종류',values='초회보험료'
     ,aggfunc=sum,fill_value=0).reset_index()

In [ ]:
# for col in df.계약종류.unique():
#     print(dfT[col])
# lis=[]
# lis.append(li)
# lis.append(li2)
round(dfT['장기']/1000)

## 실적 변동

In [ ]:
##sns 전월대비 상승,하락 사원(직영지점/사원)
#당월건 분석
plt.rcParams['axes.unicode_minus']=True
for seq, dfRow in enumerate(dfList):
    if dfRow.월[0]==thisMonth:
        dfThis1=dfRow
    elif dfRow.월[0]==prevMonth:
        dfPrev1=dfRow
    
dfT=pd.concat([dfThis1,dfPrev1])
m= dfList[0].영업일차.max()
dfT=dfT[(dfT.지점그룹=='직영') & (dfT.영업일차<=m)]

dfT=pd.pivot_table(dfT,index=['지점','수금사원명'],columns='월',values='초회보험료'
     ,aggfunc=sum,fill_value=0).reset_index()

fig,(ax1,ax2) = plt.subplots(1,2, sharey=False)
ax1= plt.subplot(221)  #top left
ax2= plt.subplot(222)  #top right
ax3= plt.subplot(212)  #bottom
plt.subplots_adjust(wspace=.8)
fig.set_size_inches(20, 10)

# 1.전월대비-지점
df1=dfT
df1['차']= (df1.iloc[:,3]-df1.iloc[:,2]) /1000
df1=df1.groupby('지점')['차'].agg('sum').reset_index().sort_values(by='차')
sns.barplot(x=df1.지점,y=df1.차,ax=ax1)
ax1.axhline(0, color="k", clip_on=False)
ax1.set_title('전월대비증감-직영',fontdict=font)
# ax1.set_xlabel("지점")
ax1.set_ylabel("실적",rotation=45)

# 2.전월대비-사원별
df1=dfT
df1['차']= (df1.iloc[:,3]-df1.iloc[:,2]) /1000
df1=df1.sort_values(by='차')
sns.barplot(x=df1.수금사원명, y=df1.차, palette="vlag", ax=ax3)
ax3.axhline(0, color="k", clip_on=False)
ax3.set_ylabel("증감",rotation=45)
ax3.set_xticklabels(df1.수금사원명, rotation=-45)
ax3.set_xlabel("전월대비증감-직영 사원별")

# 3. 지점 구성비 (당월)
plt.subplots_adjust(wspace=.8)
df2=dfList[0][dfThis1.지점그룹=='직영'].groupby(['지점'])['초회보험료'].sum().reset_index().sort_values(by='초회보험료',ascending=False).head(10)
df2['초회보험료']=df2.초회보험료 / 1000
df2['구성비']=df2.초회보험료/df2.초회보험료.sum()
ex=[0]*df2.shape[0]
ex[:3]=[0.08,0.06,0.04]  #상위1,2개 exploding
ax2.pie(df2.구성비, labels=df2.지점, autopct='%1.1f%%',
         shadow=False, startangle=45,explode=ex)
ax2.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
ax2.set_title('직영- 지점 M/S',fontdict=font)
# ax2.set_xlabel("지점")
ax2.set_ylabel("실적",rotation=45)

plt.tight_layout()
plt.savefig(outdir+'report4.png',dpi=120)
del dfThis1, dfPrev1

In [ ]:
dfT.sort_values(by='차',ascending=True)


In [ ]:
# dd=pd.concat([dfList[0],dfList[1]])
dt=pd.pivot_table(dd,index=['지점그룹','지점'],columns='월',values='초회보험료'
     ,aggfunc=np.sum,fill_value=0,dropna=True)

In [ ]:
pd.concat([
    d.append(d.sum().rename((k, 'Total')))
    for k, d in dt.groupby(level=0)
]).append(dt.sum().rename(('Grand', 'Total')))


In [ ]:
pd.concat([
    d.append(d.sum().rename((k, 'Total')))
    for k, d in dt.groupby(level=0)
])

In [ ]:
dg

In [ ]:
len(dfList[0])+len(dfList[1])+len(dfList[2])

## 지점별 가동율

In [ ]:
# df_Fc=pd.read_excel('d:/글로벌/실적분석/코드파일.xlsx',sheet_name='사원')
for seq, dfRow in enumerate(dfList):
    if dfRow.월[0]==thisMonth:
        dfThis2=dfRow
    elif dfRow.월[0]==prevMonth:
        dfPrev2=dfRow
        
df_t=df_Fc.join(dfThis2.groupby('수금사원명')['초회보험료'].agg('sum'), on='사원명')  
df_t['월']=dfThis2.loc[0,'월']
df_t1=df_Fc.join(dfPrev2.groupby('수금사원명')['초회보험료'].agg('sum'), on='사원명')  
df_t1['월']=dfPrev2.loc[0,'월']
del dfThis2, dfPrev2

df_wk=pd.concat([df_t,df_t1])
df_wk['가동']=df_wk.초회보험료.map(lambda x: 1 if x>0 else 0)
df_wk['초회보험료']=df_wk['초회보험료'].fillna(0)


import matplotlib.pyplot as plt
%matplotlib inline 

fdict={'fontsize':16, 'fontweight':20,'color':'r'}
fig,(ax1,ax2) = plt.subplots(1,2, sharey=False)
ax1= plt.subplot(221)  #top left
ax2= plt.subplot(222)  #top right
plt.subplots_adjust(wspace=.2)
fig.set_size_inches(20, 10)
dft=df_wk.groupby(['지점그룹','지점','월'])['가동'].agg(['sum','count']).reset_index()
dft.rename(columns={'sum':'가동','count':'재적'},inplace=True)
dft['가동율']= dft.가동 / dft.재적 * 100

dft1= dft[(dft.지점그룹=='직영') & (dft.월== dft.월.max())]
sns.barplot(data=dft1,x=dft1.지점, y=dft1['재적'], ax=ax1)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=-45)
ax1.set_title('재적FC',fontdict=fdict)
ax1.set_xlabel(" ")
for i,val in enumerate(dft1.재적):
    ax1.text(x=i,y=round(val),s=round(val),ha='center')

dft1= dft[dft.지점그룹=='직영']
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=-45)
ax2.set_title('가동율',fontdict=fdict)
ax2.set_ylabel("%",rotation=90)

sns.barplot(data=dft1,x=dft1.지점, y=dft1['가동율'], hue='월',  ax=ax2)
for i,val in enumerate(dft1.가동율):
    ax2.text(x=i/2,y=round(val),s=round(val),ha='left')
ax2.set_xlabel(" ")
    
plt.savefig(outdir+'report5.png',dpi=120) 

## 그래프 이미지를 합쳐서 저장

In [ ]:
from PIL import ImageSequence, Image
import glob
import sys
import os
# outdir='e:/python/5.Exercise on Glad/'

def create_aggregated_tiff(input_tiffs):
    im_list = []
    for tiff_path in input_tiffs:
#         print(tiff_path)
        img = Image.open(open(tiff_path,'rb'))
        for page in ImageSequence.Iterator(img):
            print(page)
            im_list.append(page)
#             print(len(im_list))
    print(im_list[0])
    return im_list
    #im_list.save("test.tif", compression="tiff_deflate", save_all=True)

input_tiffs = [outdir+'./report1.png',
               outdir+'./report2.png',
               outdir+'./report3.png',
               outdir+'./report4.png',
               outdir+'./report5.png'
               ]
lst = create_aggregated_tiff(input_tiffs=input_tiffs)
#     print(lst[0])
img1 = lst[0]
img1.save(outdir+"reportAll.tif", save_all=True, append_images=lst[1:])

## 당월 계약-건당보험료 상위20%

In [ ]:
import numpy as np

df=dfList[0].copy()
# bins_name=['1/4','2/4','3/4','4/4']
bins_val=[0]
bins_name=[]
count=5
for i in range(1,count+1):
    bins_name.append(str(i)+'/'+str(count))
    bins_val.append(df.초회보험료.quantile(i/count))
    
print(bins_val)
# count,bins_val=np.histogram(df.초회보험료,bins=4) # 3분할
df['등급']=pd.cut(df.초회보험료, 
            bins=bins_val,  #경계값 리스트
            labels=bins_name,  #bin이름
            include_lowest=True) #첫 경계값 포함여부

gd=[max(bins_name)]
cond=(df.등급.isin(gd)) & (df.계약일자>='2020-03-01')
cond2=(df.지점그룹=='직영')
cols=['지점','수금사원명','계약자','계약일자','초회보험료','상품종류']
df.loc[cond&cond2,cols ].sort_values(by=['지점','수금사원명'])

In [ ]:
# bins_name=['1/4','2/4','3/4','4/4']
# count,bins_val=np.histogram(df.초회보험료,bins=4) # 3분할
# df.초회보험료.rank(method='first')
# df.nlargest(100,  '초회보험료')['초회보험료']
# bins_val=[0]
# count=4
# for i in range(1,count+1):
#     print(i)
#     bins_val.append(df.초회보험료.quantile(i/count))
# # bins_val
# df.to_csv('d:/tt.csv',encoding='cp949')
bins_name.reverse()
[bins_name[len(bins_name)-1]]

## -------------------------------------위까지 마지막  
   아래는 유지율 산출..  

In [ ]:
data=pd.read_excel('d:/글로벌/실적분석/코드파일.xlsx',sheet_name='유지율파일')
df=pd.read_excel(data.폴더명[0]+data.파일명[0])
df=preProc(df,'현재월')

en='2020-03-01'
st=df.계약일자[0]
en=datetime.strptime(en,'%Y-%m-%d')
seqNo= (en.year-st.year)*12+(en.month-st.month) -2  #납입회차

#미유지건 전처리
code=['정상','부활']
code1=['철회','반송','취소','품보']
df['유지상태']=df.계약상태.map(lambda x: '정상' if x in(code) else ( '철취' if x in(code1) else '미유지'))
#2회 납입지연건은 '미유지'처리
df.loc[(df.유지상태=='정상') & (df.납입방법=='월납') & (df.납입회차< seqNo-2),'유지상태']='미유지'

dft=pd.pivot_table(df, index=['지점그룹','지점','수금사원명','계약종류'],columns='유지상태',values='초회보험료'
     ,aggfunc=sum,fill_value=0).reset_index()
dft['유지율']=(dft.정상/ (dft.정상+dft.미유지) * 100).fillna(0)
dft['철취율']=(dft.철취/ (dft.정상+dft.미유지+dft.철취) * 100).fillna(0)

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2, sharey=False)
ax1= plt.subplot(221)  #top left
ax2= plt.subplot(222)  #top right
plt.subplots_adjust(wspace=.2)
fig.set_size_inches(20, 12)

brh=['직영']
dft1=dft[dft.지점그룹.isin(brh)]
dft1=pd.pivot_table(dft1, index=['지점그룹','지점','계약종류'],values=['미유지','정상','철취']
     ,aggfunc=sum,fill_value=0).reset_index()
dft1['유지율']=(dft1.정상/ (dft1.정상+dft1.미유지) * 100).fillna(0)
dft1['철취율']=(dft1.철취/ (dft1.정상+dft1.미유지+dft1.철취) * 100).fillna(0)
sns.barplot(data=dft1,x=dft1.지점, y=dft1['유지율'],hue='계약종류', ax=ax1)
sns.lineplot(data=dft1,x=dft1.지점, y=dft1['철취율'],ax=ax1)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=-45)
ax1.set_title('13회차 유지율')

dft1=dft[(dft.지점그룹.isin(brh)) & (dft.계약종류=='장기')].sort_values('유지율').head(20)
sns.barplot(data=dft1,x=dft1.수금사원명, y=dft1['유지율'],hue='계약종류', ax=ax2)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=-45)
ax2.set_title('유지율 bottom20')

In [ ]:
p=pd.Period(2019,freq='A-DEC')

In [ ]:
data.최종상태변경일=pd.to_datetime(data.최종상태변경일,format='%Y-%m-%d')

In [ ]:
pd.date_range('1/1/2019','6/30/2019',freq='m')
# rng=pd.period_range('1/1/2019','6/30/2019',freq='d')

In [ ]:
import pandas as pd
import mypkgs.MyDateUtil as MyU  #자체 유틸리티 클래스 로딩
from datetime import date, datetime
df=pd.read_excel('e:/python/testData/test.xlsx')

In [ ]:
'시흥' in ('엠피시1')

In [ ]:
#--그래프에 값을 표시하는 사례
# barh 함수(bar 위치의 배열, 구체적인 수치의 배열, algin='center', height=0.5)를 통해 각 인덱스와 값을 입력
rects = plt.barh(ind, diff.sort_values()[:20].values, align='center', height=0.5)
# yticks 함수를 통해 각 인덱스와 이름 태그를 입력
plt.yticks(ind, diff.sort_values()[:20].index)
# 막대마다 수치를 달아주는 작업
for i, rect in enumerate(rects):
    # text 함수(값이 출력되는 X 축의 위치, 값이 출력되는 Y 축의 위치, 해당 위치에 출력할 값을 전달)를 통해 수치 매핑
    ax.text(0.95 * rect.get_width(),
            rect.get_y() + rect.get_height() / 2.0,
            str(round(diff.sort_values()[:20].values[i], 2)) + '%',
            ha='left', va='center', bbox=dict(boxstyle="round", fc=(0.5, 0.9, 0.7), ec="0.1"))
            
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

f, axes = plt.subplots(2, 10, sharex=True, gridspec_kw={"height_ratios":(.35, .35)}, 
                                    figsize = (12, 6))

print(f.get_size_inches)

df = pd.DataFrame(np.random.randint(0,100,size=(100, 10)),columns=list('ABCDEFGHIJ'))

for i, c in enumerate(df):
    sns.boxplot(df[c], ax=axes[0,i])
    axes[1,i].hist(df[c])
plt.tight_layout()
plt.show()
